# Isochrones with Python and APIS

In [1]:
# We'll use this to break our address into street, city, state pieces for us
import usaddress

# Lots of meetings are at the Cook County Building.  This is one example from the data.
location_name = "Cook County Building, Board Room, 118 North Clark Street, Chicago, Illinois"

# Many geocoders, especially free ones need us to break the address into pieces.
# Let's use the awesome usaddress package to do this for us
parsed_location_name, detected_type = usaddress.tag(location_name)
print(parsed_location_name)

OrderedDict([('BuildingName', 'Cook County Building, Board Room'), ('AddressNumber', '118'), ('StreetNamePreDirectional', 'North'), ('StreetName', 'Clark'), ('StreetNamePostType', 'Street'), ('PlaceName', 'Chicago'), ('StateName', 'Illinois')])


In [2]:
# Let's merge these very atomic fields into ones that are closer to those
# accepted by many geocoding APIs
street_address = ' '.join([
    parsed_location_name['AddressNumber'],
    parsed_location_name['StreetNamePreDirectional'],
    parsed_location_name['StreetName'],
    parsed_location_name['StreetNamePostType']
])
address = {
    'street_address': street_address,
    'city': parsed_location_name['PlaceName'],
    'state': parsed_location_name['StateName']
}
print(address)

{'street_address': '118 North Clark Street', 'city': 'Chicago', 'state': 'Illinois'}


In [3]:
import csv
import io

# We'll use this to simplify making our HTTP requests to the geocoding APIs
import requests

# Let's merge our parsed address into fields that are

# We'll use the Texas A&M Geoservices API because it's free (for few requests)
# and has pretty liberal terms of service.  It does however require an API
# key.
TAMU_GEOSERVICES_API_KEY = os.environ.get('TAMU_GEOSERVICES_API_KEY')

api_url_tpl = (
    "https://geoservices.tamu.edu/Services/Geocode/WebService/GeocoderWebServiceHttpNonParsed_V04_01.aspx?"
    "apiKey={api_key}&"
    "version={version}&"
    "streetAddress={street_address}&"
    "city={city}&"
    "state={state}&"
    "includeHeader=true"
)
version = '4.01'

api_url = api_url_tpl.format(
    api_key=TAMU_GEOSERVICES_API_KEY,
    version=version,
    street_address=address['street_address'],
    city=address['city'],
    state=address['state'],
)

r = requests.get(api_url)

inf = io.StringIO(r.text)

geocoded = None
reader = csv.DictReader(inf)

for row in reader:
    geocoded = row
    # Only read one row
    break
    
print(geocoded)

OrderedDict([('TransactionId', '4bae041a-570d-4348-b05e-66623185ca7d'), ('Version', '4.1'), ('QueryStatusCodeValue', '200'), ('Latitude', '41.8834456872408'), ('Longitude', '-87.631003848859'), ('NAACCRGISCoordinateQualityCode', '03'), ('NAACCRGISCoordinateQualityName', 'StreetSegmentInterpolation'), ('MatchScore', '96.0059171597633'), ('MatchType', 'Relaxed'), ('FeatureMatchingResultType', 'Success'), ('FeatureMatchingResultCount', '1'), ('FeatureMatchingGeographyType', 'StreetSegment'), ('RegionSize', '3174.85431915501'), ('RegionSizeUnits', 'Meters'), ('MatchedLocationType', 'LOCATION_TYPE_STREET_ADDRESS'), ('TimeTaken', '0.0390039'), ('', '')])


In [4]:
# Our longitude and latitude are available as keys of the parsed dictionary
print("[{0}, {1}]".format(geocoded['Longitude'], geocoded['Latitude']))

[-87.631003848859, 41.8834456872408]


In [5]:
from datetime import datetime

from pytz import timezone

start_time = "March 8, 2017 5:00pm"

parsed_start_time = datetime.strptime(start_time, "%B %d, %Y %I:%M%p")

# Convert to Chicago's time zone
central = timezone('US/Central')
parsed_start_time = central.localize(parsed_start_time)

print(parsed_start_time.isoformat())

2017-03-08T17:00:00-06:00


In [6]:
import os

import requests

# Let's get an isochrone using the HERE Routing API Isoline endpoint

HERE_APP_ID = os.environ.get('HERE_APP_ID')
HERE_APP_CODE = os.environ.get('HERE_APP_CODE')

api_params = {
    'app_id': HERE_APP_ID,
    'app_code': HERE_APP_CODE,
    'mode': "fastest;car;traffic:enabled",
    'destination': "geo!{lat},{lng}".format(lat=geocoded['Latitude'], lng=geocoded['Longitude']),
    'arrival': parsed_start_time.isoformat(),
    # 1 hour
    'range': 1 * 60 * 60,
    'range_type': 'time'
}
api_url = (
    "https://isoline.route.cit.api.here.com/routing/7.2/calculateisoline.json"
    "?app_id={app_id}"
    "&app_code={app_code}"
    "&mode={mode}"
    "&destination={destination}"
    "&range={range}"
    "&rangetype={range_type}"
).format(**api_params)

r = requests.get(api_url)

isoline_response = r.json()['response']
print(isoline_response)

{'metaInfo': {'timestamp': '2017-10-13T01:15:37Z', 'mapVersion': '8.30.73.154', 'moduleVersion': '7.2.201739-162062', 'interfaceVersion': '2.6.34', 'availableMapVersion': ['8.30.73.154']}, 'center': {'latitude': 41.8834456, 'longitude': -87.6310039}, 'isoline': [{'range': 3600, 'component': [{'id': 0, 'shape': ['41.8029785,-88.3287048', '41.8029785,-88.3081055', '41.8057251,-88.2998657', '41.8222046,-88.2833862', '41.8222046,-88.2778931', '41.8057251,-88.2723999', '41.8057251,-88.2669067', '41.8112183,-88.2614136', '41.8167114,-88.2449341', '41.8222046,-88.2394409', '41.8276978,-88.2229614', '41.8331909,-88.2229614', '41.8386841,-88.2284546', '41.8441772,-88.2284546', '41.8496704,-88.2119751', '41.8661499,-88.1954956', '41.8688965,-88.1872559', '41.8688965,-88.1433105', '41.8716431,-88.1350708', '41.8826294,-88.1240845', '41.8881226,-88.1240845', '41.8936157,-88.140564', '41.9046021,-88.1515503', '41.9100952,-88.1515503', '41.9155884,-88.1350708', '41.9238281,-88.1323242', '41.9320679,

In [7]:
# Convert the isoline API response to GeoJSON
import json

isoline_geojson = {
    'type': 'FeatureCollection',
    'features': [],
}

isoline_geojson['features'].append({
    'type': 'Feature',
    'geometry': {
        'type': 'Point',
        'coordinates': [
            isoline_response['center']['longitude'],
            isoline_response['center']['latitude']
        ],
    },
    'properties': {},
})

isoline_geojson['features'].append({
    'type': 'Feature',
    'geometry': {
        'type': 'Polygon',
        'coordinates': [
            [
                [float(c) for c in x.split(',')[::-1]]
                for x
                in isoline_response['isoline'][0]['component'][0]['shape']
            ],
        ],
    },
    'properties': {},
})


In [8]:
# And display it using geojson.io

import geojsonio
geojsonio.display(json.dumps(isoline_geojson))

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Point%22%2C%20%22coordinates%22%3A%20%5B-87.6310039%2C%2041.8834456%5D%7D%2C%20%22properties%22%3A%20%7B%7D%7D%2C%20%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Polygon%22%2C%20%22coordinates%22%3A%20%5B%5B%5B-88.3287048%2C%2041.8029785%5D%2C%20%5B-88.3081055%2C%2041.8029785%5D%2C%20%5B-88.2998657%2C%2041.8057251%5D%2C%20%5B-88.2833862%2C%2041.8222046%5D%2C%20%5B-88.2778931%2C%2041.8222046%5D%2C%20%5B-88.2723999%2C%2041.8057251%5D%2C%20%5B-88.2669067%2C%2041.8057251%5D%2C%20%5B-88.2614136%2C%2041.8112183%5D%2C%20%5B-88.2449341%2C%2041.8167114%5D%2C%20%5B-88.2394409%2C%2041.8222046%5D%2C%20%5B-88.2229614%2C%2041.8276978%5D%2C%20%5B-88.2229614%2C%2041.8331909%5D%2C%20%5B-88.2284546%2C%2041.8386841%5D%2C%20%5B-88.2284546%2C%2041.8441772%5D%2C%20%5B-88.2119751%

In [9]:
# Factor this example into functions so we can get multiple areas

import requests

def get_isoline(lat, lng, mode, rng, app_id, app_code):
    api_params = {
        'app_id': app_id,
        'app_code': app_code,
        'mode': mode,
        'destination': "geo!{lat},{lng}".format(lat=lat, lng=lng),
        'arrival': parsed_start_time.isoformat(),
        # 1 hour
        'range': rng,
        'range_type': 'time'
    }
    api_url = (
        "https://isoline.route.cit.api.here.com/routing/7.2/calculateisoline.json"
        "?app_id={app_id}"
        "&app_code={app_code}"
        "&mode={mode}"
        "&destination={destination}"
        "&range={range}"
        "&rangetype={range_type}"
    ).format(**api_params)

    r = requests.get(api_url)

    return r.json()['response']

def isoline_geojson_feature(isoline):
    return {
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
            'coordinates': [
                [
                    [float(c) for c in x.split(',')[::-1]]
                    for x
                    in isoline_response['isoline'][0]['component'][0]['shape']
                ],
            ],
        },
        'properties': {},
    }



In [10]:
ranges = [
    {
        'range': 1 * 60 * 60,
        'properties': {
            'fill': '#e5f5f9',
        },
    },
    {
        'range': 1 * 60 * 30,
        'properties': {
            'fill': '#e5f5f9',
        },
    },
]

isolines = {
    'type': 'FeatureCollection',
    'features': [],
}

isolines['features'].append({
    'type': 'Feature',
    'geometry': {
        'type': 'Point',
        'coordinates': [
            float(geocoded['Longitude']),
            float(geocoded['Latitude']), 
        ],
    },
    'properties': {},
})

for rng in ranges:
    isoline = get_isoline(
        geocoded['Latitude'], 
        geocoded['Longitude'], 
        'fastest;car;traffic:enabled',
        rng['range'],
        HERE_APP_ID,
        HERE_APP_CODE
    )

    isoline_feature = isoline_geojson_feature(isoline)
    isoline_feature['properties'].update(**rng['properties'])
    isoline_feature['properties']['range'] = rng['range']
    isolines['features'].append(isoline_feature)

In [11]:
# And display it using geojson.io

import geojsonio

geojsonio.display(json.dumps(isolines))

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Point%22%2C%20%22coordinates%22%3A%20%5B-87.631003848859%2C%2041.8834456872408%5D%7D%2C%20%22properties%22%3A%20%7B%7D%7D%2C%20%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Polygon%22%2C%20%22coordinates%22%3A%20%5B%5B%5B-88.3287048%2C%2041.8029785%5D%2C%20%5B-88.3081055%2C%2041.8029785%5D%2C%20%5B-88.2998657%2C%2041.8057251%5D%2C%20%5B-88.2833862%2C%2041.8222046%5D%2C%20%5B-88.2778931%2C%2041.8222046%5D%2C%20%5B-88.2723999%2C%2041.8057251%5D%2C%20%5B-88.2669067%2C%2041.8057251%5D%2C%20%5B-88.2614136%2C%2041.8112183%5D%2C%20%5B-88.2449341%2C%2041.8167114%5D%2C%20%5B-88.2394409%2C%2041.8222046%5D%2C%20%5B-88.2229614%2C%2041.8276978%5D%2C%20%5B-88.2229614%2C%2041.8331909%5D%2C%20%5B-88.2284546%2C%2041.8386841%5D%2C%20%5B-88.2284546%2C%2041.8441772%5D%2C%20%5B-

In [ ]:
# Now let's try to draw an isochrone using transit data

